# Project2_Taewoo Kim_20233219

In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
import os, string, spacy, gensim, nltk
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer
from gensim.corpora.dictionary import Dictionary
from gensim import models
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora, models, utils
from gensim.parsing.preprocessing import remove_stopwords
nlp = spacy.load('en_core_web_sm')
nlp.max_length = 1000000
from nltk.tokenize import sent_tokenize
import re

import warnings
warnings.filterwarnings(action="ignore")

* Preprocessing

In [2]:
NLP_finn = pd.read_csv('pjt2_data/NLP_finn.csv')
# creating a corpus with all abstracts
corpus = [NLP_finn['Abst'][i] for i in range(len(NLP_finn['Abst']))]

# cleaning the unnecessary terms and creating a cleaned corpus
corpusn = [i.replace('Research Summary', '').replace('Research Abstract', '').replace('Research summary', '') for i in corpus]

corpusnn = []
for i in corpusn:
    sentences = sent_tokenize(i)
    for j in sentences:
        if 'Copyright (' in j:
            sentences.remove(j)
    cleanedAbst = ' '.join(sentences) # combine strings with ' '
    corpusnn.append(cleanedAbst)

# Removing Punctuations
corpus_P = [i.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))).replace(' '*4, ' ').replace(' '*3, ' ').replace(' '*2, ' ').strip() for i in corpusnn]

# Removing Numbers
corpus_PN = [i.translate(str.maketrans('', '', string.digits)) for i in corpus_P]

# Lowercasing
corpus_PNL = [i.lower()  for i in corpus_PN]

# Lemmatizing
#corpus_PNLM = [[j.lemma_ for j in nlp(i)] for i in corpus_PNL]
# removing empty strings from the results of lemmatizing
#corpus_PNLMf = [' '.join(i).split() for i in corpus_PNLM]
import pickle
#with open('pjt2_data/corpus_PNLMf.pickle', 'wb') as f:
#    pickle.dump(corpus_PNLMf, f)
with open('pjt2_data/corpus_PNLMf.pickle', 'rb') as f:
    corpus_PNLMf = pickle.load(f)

# Removing Stopwords
all_stopwords = gensim.parsing.preprocessing.STOPWORDS
all_stopwords_add = all_stopwords.union(set(['x', 'y', 'I', 's', 'study', 'aaa', 'z', 'ab', 'abd', 'yes', 'year',
                                        'zero', 'writing', 'd','r']))

corpus_PNLMW = [[j for j in i if not j in all_stopwords_add] for i in corpus_PNLMf]
corpus_PNLMWs = [' '.join(i) for i in corpus_PNLMW]

In [3]:
# finding optimal min_df, max_df
def feed(wordlist):
    """Use this for feeding preprocessed tokens"""
    return wordlist

# min_df와 max_df 범위 설정
min_df_range = [2, 3, 4, 5, 6, 7, 8]
max_df_range = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4]

# 단어 개수의 변화율을 파악하여 최적의 매개변수 결정
opt_min_df = None
opt_max_df = None
cv_init = CountVectorizer(tokenizer=feed, min_df=2, max_df=0.9, ngram_range=(1, 1), token_pattern=None, lowercase=False)
X_init = cv_init.fit_transform(corpus_PNLMW)
init_word_count = X_init.sum()
threshold_word_count = init_word_count * 0.95
results = []
for min_df in min_df_range:
    for max_df in max_df_range:
        #print(min_df, max_df)
        cv = CountVectorizer(tokenizer=feed, min_df=min_df, max_df=max_df, ngram_range=(1, 1), token_pattern=None, lowercase=False)
        X = cv.fit_transform(corpus_PNLMW)
        word_count = X.sum()
        if word_count <= threshold_word_count:
            results.append((min_df, max_df, word_count))
            break
    if len(results) > 0:
        break
print(f"base Word Count: {init_word_count}")
if len(results) > 0:
    opt_min_df, opt_max_df, opt_word_count = results[0]
    print(f"opt min_df: {opt_min_df}, opt max_df: {opt_max_df}, opt Word Count: {opt_word_count}")
else:
    print("No suitable combination of min_df and max_df was found.")

base Word Count: 1031182
opt min_df: 8, opt max_df: 0.4, opt Word Count: 977928


In [4]:
# Making document-term frequency matrix
P_names = ['p' + str(i) for i in range(len(corpus_PNLMW))]

dtm_md = CountVectorizer(tokenizer = feed, min_df = opt_min_df, max_df = opt_max_df, ngram_range = (1,3), token_pattern = None, lowercase = False)
dtfm = dtm_md.fit_transform(corpus_PNLMW)
print(type(dtfm), dtfm.shape)

dtfmx = pd.DataFrame(dtfm.toarray(), index= P_names, columns= dtm_md.get_feature_names_out())
print(dtfmx.head())

# DTM to DTL by document
fin_words = dtm_md.inverse_transform(dtfm)
print(len(fin_words[0]), fin_words[0])

# Mapping between unique words and word id (making a dictionary)
pp_dict = Dictionary(fin_words)

# Given a dictionary (pp_dict), make a word frequency table for each document.
wordfreq_doc = [pp_dict.doc2bow(text) for text in fin_words] #corpus_PNLMW]
print(len(wordfreq_doc))
print(wordfreq_doc[0], end = " ")

<class 'scipy.sparse._csr.csr_matrix'> (11231, 17444)
    abandon  abandonment  abductive  ability  ability adapt  ability build   
p0        0            0          0        0              0              0  \
p1        0            0          0        0              0              0   
p2        0            0          0        0              0              0   
p3        0            0          0        0              0              0   
p4        0            0          0        0              0              0   

    ability firm  ability generate  ability identify  ability learn  ...   
p0             0                 0                 0              0  ...  \
p1             0                 0                 0              0  ...   
p2             0                 0                 0              0  ...   
p3             0                 0                 0              0  ...   
p4             0                 0                 0              0  ...   

    yield important 

## LSA

In [5]:
n_topics = 50
n_words = 10
topics = ['topic'+ str(i+1) for i in range(n_topics)]
pp_LSA_sklm = TruncatedSVD(n_components= n_topics, n_iter=7, random_state=42)
pp_LSA_dtm = pp_LSA_sklm.fit_transform(dtfm)
pp_topictermMtx = pd.DataFrame(pp_LSA_sklm.components_, index = topics, columns= dtm_md.get_feature_names_out()) # relationship between topics and terms
idx_nmax_rf = [abs(row).nlargest(n_words).index for index, row in pp_topictermMtx.iterrows()] # list of top 10 words for each topic

In [6]:
lsa_topics_words = pd.DataFrame(idx_nmax_rf).T
lsa_topics_words.columns = topics
lsa_topics_words

,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,...,topic41,topic42,topic43,topic44,topic45,topic46,topic47,topic48,topic49,topic50
0,performance,product,model,performance,team,performance,market,knowledge,knowledge,risk,...,base,technology,consumer,theory,contract,strategic,status,research,board,relationship
1,model,performance,product,team,product,product,team,performance,effect,market,...,manager,innovation,increase,technology,increase,experience,manager,board,director,increase
2,effect,price,market,knowledge,risk,team,model,alliance,information,product,...,platform,relationship,relationship,datum,innovation,business,experience,platform,contract,process
3,market,team,problem,organization,market,information,customer,effect,organization,decision,...,consumer,business,social,manager,technology,strategy,business,increase,employee,research
4,product,model,time,organizational,ceo,model,risk,organization,network,customer,...,supplier,experience,innovation,process,work,platform,innovation,director,social,work
5,result,cost,effect,new,knowledge,knowledge,cost,social,model,information,...,strategy,strategy,level,status,supplier,organization,result,project,quality,base
6,high,consumer,decision,high,member,price,performance,resource,organizational,knowledge,...,work,platform,manager,contract,theory,organizational,increase,ceo,ceo,supplier
7,increase,organizational,high,effect,consumer,supplier,service,cost,decision,service,...,policy,investment,design,time,board,management,service,time,organization,innovation
8,base,market,industry,product,value,cost,supplier,problem,customer,price,...,innovation,research,organizational,quality,venture,learn,acquisition,theory,status,business
9,level,organization,consumer,process,customer,network,employee,consumer,alliance,ceo,...,time,status,quality,new,service,industry,employee,datum,relationship,employee


## LDA

In [7]:
lda_model = models.ldamodel.LdaModel(corpus=wordfreq_doc, num_topics=n_topics, id2word=pp_dict, passes=15,
                                    alpha='auto', update_every=1, chunksize=64, random_state=100)
#lda_topics = lda_model.print_topics(num_topics=n_topics, num_words=n_words)

In [8]:
lda_list = []
for topic_id in range(n_topics):
    topic_words = []
    for word, prob in lda_model.show_topic(topic_id, topn=n_words):
        topic_words.append(word)
    lda_list.append(topic_words)

In [9]:
print(lda_model.show_topic(0,topn=10))

[('likelihood', 0.2540049), ('target', 0.23612055), ('chief', 0.07159696), ('chief executive', 0.06826156), ('tenure', 0.06553857), ('target firm', 0.060748048), ('existence', 0.050175615), ('curvilinear', 0.030064043), ('low likelihood', 0.015859047), ('great level', 0.010059567)]


In [10]:
lda_topics_words = pd.DataFrame(lda_list).T
lda_topics_words.columns = topics
lda_topics_words

,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,...,topic41,topic42,topic43,topic44,topic45,topic46,topic47,topic48,topic49,topic50
0,likelihood,variable,manager,public,simulation base,future,little,demonstrate,financial,simulation base,...,introduce,argument,approach,competition,help,long,account,view,knowledge,position
1,target,age,insight,uncertainty,complex organizational,shift,know,need,significant,complex organizational,...,field,contrast,method,policy,exist,term,behavioral,key,importance,relevant
2,chief,education,moderate,strategic management,gets,combine,finding suggest,differ,evaluate,gets,...,setting,post,assess,rate,consistent,incentive,self,dynamic,critical,hire
3,chief executive,disappear,theorize,environmental,firm difficult,occur,attribute,boundary,productivity,firm difficult,...,conduct,contingent,relation,relatively,enable,stock,constraint,specific,number,government
4,tenure,theoretical model,furthermore,political,organization different,trigger,generally,define,financial performance,organization different,...,qualitative,performance firm,cognitive,encourage,service,ownership,methodological,especially,require,distinguish
5,target firm,income,peer,treat,disappointment,review,situation,responsibility,region,disappointment,...,initial,robust,distance,fail,vary,long term,health,reveal,direct,directly
6,existence,stereotype,diverse,collect,task perform,poorly,stable,result indicate,performance result,task perform,...,close,shape relationship,guide,attempt,dimension,short,implication result,link,increasingly,employment
7,curvilinear,occupational,antecedent,dependence,allege,initiate,little know,commitment,result base,allege,...,line,turnover,reputation,tendency,sector,officer,result discuss,good,success,senior
8,low likelihood,questionnaire,category,american,organizational task,suggestion,stability,broadly,firm performance result,organizational task,...,quantitative,u shape relationship,similarity,law,ideal,high performance,self report,compete,scholar,actual
9,great level,allege,moderate effect,partial,potential new,analytic,error,clearly,thousand,potential new,...,professional,sample firm,conventional,implementation,profile,short long,organization different,second,cause,promotion


## Bertopic

In [11]:
BERTopic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True, top_n_words=n_words, nr_topics=n_topics)
BERT_topics, BERT_probs = BERTopic_model.fit_transform(corpus_PNLMWs)

Batches:   0%|          | 0/351 [00:00<?, ?it/s]

2023-05-01 19:22:24,730 - BERTopic - Transformed documents to Embeddings
2023-05-01 19:22:51,969 - BERTopic - Reduced dimensionality
2023-05-01 19:23:05,212 - BERTopic - Clustered reduced embeddings
2023-05-01 19:23:09,525 - BERTopic - Reduced number of topics from 150 to 50


In [12]:
# Get top words for each topic
BERTopic_list = []
for topic_id in range(-1, n_topics-1):
    topic_words=[]
    for i in range(n_words): 
        topic_words.append(BERTopic_model.get_topic(topic_id)[i][0])
    BERTopic_list.append(topic_words)

In [13]:
BERTopic_topics_words = pd.DataFrame(BERTopic_list).T
BERTopic_topics_words.columns = topics
BERTopic_topics_words

,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,...,topic41,topic42,topic43,topic44,topic45,topic46,topic47,topic48,topic49,topic50
0,firm,team,price,service,fund,ceo,problem,risk,venture,foreign,...,franchise,stigma,imitation,divestiture,licensing,agglomeration,cultural,audit,aspiration,rent
1,use,organizational,retailer,customer,return,board,algorithm,option,entrepreneur,country,...,franchisee,stigmatize,imitate,divestment,licensee,geographic,industry,auditor,performance,creation
2,performance,organization,supplier,employee,portfolio,director,solution,utility,entrepreneurial,subsidiary,...,franchisor,disapproval,imitator,family,license,location,culture,sox,feedback,resource
3,model,identity,inventory,job,risk,ceos,solve,decision,entrepreneurship,multinational,...,franchising,category,original,divest,technology,industry,tradition,big,level,mechanism
4,product,process,consumer,queue,stock,family,procedure,preference,founder,mnc,...,advertising,stigmatization,imitative,firm,licensed,cluster,element,auditing,search,retrenchment
5,result,performance,product,work,analyst,firm,method,probability,startup,firm,...,outlet,audience,leader,divester,clause,geographically,artist,inspectee,behavioral,foresight
6,market,learning,auction,worker,investor,executive,time,choice,new,mne,...,fee,organization,creative,dispute,competition,establishment,museum,quality,firm,capability
7,effect,firm,demand,performance,asset,compensation,programming,uncertainty,institutional,host,...,chain,industry,rival,non,innovation,locate,art,reporting,attention,appropriation
8,high,industry,buyer,turnover,volatility,pay,scheduling,method,vcs,international,...,franchised,attack,practice,undertake,payment,firm,producer,audits,theory,ricardian
9,organization,change,cost,time,market,performance,heuristic,function,entry,investment,...,store,input,copy,business,royalty,internal,creative,deficiency,historical,theory


In [14]:
# randomly pick five topics per model
idx_lsa = [f'topic{n}' for n in np.random.choice(n_topics, 5, replace=False)]
idx_lda = [f'topic{n}' for n in np.random.choice(n_topics, 5, replace=False)]
idx_BERTopic = [f'topic{n}' for n in np.random.choice(n_topics, 5, replace=False)]

columns = []
for i in ['LSA', 'LDA', 'BERTopic']:
    for j in range(5):
        columns.append(i+ str(j+1))

df_lsa = lsa_topics_words.loc[:, idx_lsa]
df_lda = lda_topics_words.loc[:, idx_lda]
df_BERTopic = BERTopic_topics_words.loc[:, idx_BERTopic]
df = pd.concat([df_lsa, df_lda, df_BERTopic], axis=1)
df.columns = columns       

In [15]:
df



,LSA1,LSA2,LSA3,LSA4,LSA5,LDA1,LDA2,LDA3,LDA4,LDA5,BERTopic1,BERTopic2,BERTopic3,BERTopic4,BERTopic5
0,status,alliance,performance,technology,process,manager,pay,simulation base,new,ceo,journal,imitation,stakeholder,patch,flexibility
1,manager,decision,model,innovation,quality,insight,selection,complex organizational,base,firm performance,scientist,imitate,csr,piracy,ambidexterity
2,experience,resource,effect,relationship,investment,moderate,emphasize,gets,develop,perspective,academic,imitator,firm,vendor,capability
3,business,strategy,market,business,time,theorize,bias,firm difficult,management,executive,research,original,social,copyright,exploitation
4,innovation,market,product,experience,technology,furthermore,compensation,organization different,focus,board,publication,imitative,corporate,security,exploration
5,result,information,result,strategy,value,peer,committee,disappointment,business,establish,science,leader,responsibility,software,dynamic
6,increase,ceo,high,platform,search,diverse,gets,task perform,article,ceos,article,creative,csp,vulnerability,flexible
7,service,partner,increase,investment,platform,antecedent,sentiment,allege,create,agency,publish,rival,sustainability,release,resource
8,acquisition,supplier,base,research,industry,category,firm difficult,organizational task,offer,director,management,practice,value,digital,organizational
9,employee,consumer,level,status,relationship,moderate effect,time market,potential new,research,monitoring,replication,copy,medium,content,firm
